In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
import joblib

# Tải mô hình Random Forest
rf_model = joblib.load("model_randomforest.joblib")

# Tải mô hình trích xuất đặc trưng (MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,
    pooling='avg',
    weights='imagenet'
)

# Thêm lớp Dense để giảm số chiều của embedding xuống 512
x = tf.keras.layers.Dense(512, activation='relu')(base_model.output)
feature_extractor = tf.keras.Model(inputs=base_model.input, outputs=x)

# Lấy danh sách tên lớp
train_dir = "C:/Users/Loc/Desktop/DACS_git/Do_An_Co_So/code/saved_embeddings"
class_names = sorted(os.listdir(train_dir))

# Kích thước ảnh đầu vào
IMG_SIZE = (160, 160)

# Tải Haar Cascade để phát hiện khuôn mặt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Mở webcam
cap = cv2.VideoCapture(0)
print("[INFO] Webcam đang mở. Nhấn 'q' để thoát.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        try:
            face_resized = cv2.resize(face, IMG_SIZE)
        except:
            continue

        face_normalized = face_resized / 255.0
        face_input = np.expand_dims(face_normalized, axis=0)

        # Trích xuất đặc trưng (embedding) với 512 chiều
        embedding = feature_extractor.predict(face_input)

        # Dự đoán bằng Random Forest
        pred_class = rf_model.predict(embedding)[0]  # Đảm bảo lấy giá trị đầu tiên trong mảng kết quả

        # Kiểm tra kiểu dữ liệu của pred_class
        if isinstance(pred_class, np.ndarray):
            pred_class = pred_class.item()  # Chuyển mảng về giá trị đơn

        # Kiểm tra xem pred_class có phải là một số nguyên hay không
        if not isinstance(pred_class, int):
            print(f"Lỗi: pred_class không phải là : {pred_class}")
            continue

        label = class_names[pred_class]

        # Vẽ kết quả
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    cv2.imshow("Face Recognition (Random Forest)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Webcam đang mở. Nhấn 'q' để thoát.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Lỗi: pred_class không phải là : Linh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Lỗi: pred_class không phải là : Sang
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Lỗi: pred_class không phải là : Linh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Lỗi: pred_class không phải là : Linh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Lỗi: pred_class không phải là : Sang
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Lỗi: pred_class không phải là : Khôi
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Lỗi: pred_class không phải là : Linh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Lỗi: pred_class không phải là : Sang
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Lỗi: pred_class không phải là : Sang
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Lỗi: pred_class không phải là : Sang
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Lỗi: pred_class không phải là : Linh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Lỗi: pred_class không phải là : Linh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Lỗi: pred_class khô